# Guide for a better C++

## Todo

- [x] Example with string
- [x] Example with matrices
- [x] Eager Evaluation
- [x] Return value optimization
- [x] Cost of virtual functions
- [x] Virtualizing non member functions
- [x] Requiring or prohibiting heap-based objects
- [x] Multiple Dispatching
- [x] Finish More Effective C++
- [ ] Difference between const and constexpr with assmebler explanation
- [ ] Check Combine C++ and C (for personnal purpose not for the guide)
- [ ] Check C++17 specs
- [ ] Check that all snippet of codes compile


**Includes**

In [1]:
#include <iostream>

## Topics to deal with

### C++ Tips

#### <ins>Lazy evaluation:</ins>

**Principle: Do not evaluate an expression until the results is required.**

This method might save computations whose results are not used.
Here are 4 example of lazy evaluation application areas:

- **Reference counting:** Count references to objects instead of always copying data.
- **Distinguish Reads from Write:** When using reference counting, create new copie of data only when using *operator[]* to write.
- **Lazy Fetching:** When dealing with large objects, read no data from object until a specific attribute is asked for Read/Write.
- **Lazy Evaluation:** When making operations on structure, store the operation in a structure and defer computation until result is required.

##### <ins>Example 1:</ins>

In [2]:
std::string s1 = "Hello";
std::string s2 = s1; //Calls string copy ctor which performs an allocation 
                //then a strcopy to copy data
                //(s1 and s2 have their own copy of the value)

printf("Address of s1 data: %p\n", s1.data());
printf("Address of s2 data: %p\n", s2.data());

Address of s1 data: 0x7f6be8ab4040
Address of s2 data: 0x7f6be8ab4060


This is how data is stored, however it is not needed to have data duplicated until s2 is modified.

<div style="text-align:center"><img src="https://i.imgur.com/zxaUBIm.png" /></div>

One method is to implement a struct nested in String that holds a *referenceCounter* and avoid data duplication until it is really needed.
Here is a graphical representation of this model:


<div style="text-align:center"><img src="https://i.imgur.com/Ov7zKM4.png" /></div>

Snippet of code representing this model.

In [3]:
class String
{
    public:
        struct StringValue {
            //Init data and strcopy initValue inside
            //Set refCount to 1
            StringValue(const char* initValue)
            : m_pData(new char[strlen(initValue)]),
              m_sRefCount(1)
            {
            }

            //Delete data
            ~StringValue() { delete[] m_pData; }

            size_t m_sRefCount;
            char* m_pData;        
        };


        String(const char* data="")
        : m_pValue(new StringValue(data))
        {    
        }

        //Destroy StringValue
        ~String()
        {
            delete m_pValue;
        }

        //Copy constructor => Do the same for operator=
        String(const String& rhs)
        {
            std::cout << "COPY CTOR" << std::endl;
            if (m_pValue == rhs.m_pValue)    return;

            //By doing this, we postpone data duplication
            // data evaluation consisting on: creating a new char[] and calling strcopy
            if (m_pValue && --m_pValue->m_sRefCount == 0)    delete m_pValue;

            m_pValue = rhs.m_pValue;
            m_pValue->m_sRefCount++;
        }

        //Evaluation of duplication operation
        void EvaluateDuplication()
        {
            //Decrease reference counter
            m_pValue->m_sRefCount--;    

            //Creates its own StringValue
            m_pValue = new StringValue(m_pValue->m_pData);
        }

        //Writing operation: data duplication must be evaluated
        void ToLowerCase()
        {
            if (m_pValue->m_sRefCount > 1)
                EvaluateDuplication();

            //... LowerCase operation
        }

        StringValue* m_pValue;
};

In [9]:
String S1("Hello");
String S2 = S1;

//show data addr
printf("Reference count of S1 Value: %zu\n", S1.m_pValue->m_sRefCount);
printf("Address of S1 data: %p\n", S1.m_pValue->m_pData);
printf("Address of S2 data: %p\n", S2.m_pValue->m_pData);

//Here let's make a modification on S2
S2.ToLowerCase();
std::cout << "\nPerformed modification on S2 !\n\n";

//We can see its address is now different also ref count changed
printf("Reference count of S1 Value: %zu\n", S1.m_pValue->m_sRefCount);
printf("Address of S2 data: %p\n", S2.m_pValue->m_pData);

COPY CTOR
Reference count of S1 Value: 2
Address of S1 data: 0x55defe434ed0
Address of S2 data: 0x55defe434ed0

Performed modification on S2 !

Reference count of S1 Value: 1
Address of S2 data: 0x55defe42ad10


##### <ins>Example 2:</ins>

Here is a class representing a 2D vector:

In [15]:
struct Vec2
{
    float m_x;
    float m_y;
    
    Vec2(float x, float y)
    : m_x(x),
      m_y(y)
    {}
    
    Vec2 operator+(const Vec2& rhs) const
    {
        return Vec2(m_x + rhs.m_x, m_y + rhs.m_y);
    }
    
    float dot(const Vec2& rhs)
    {
        return m_x * rhs.m_x + m_y * rhs.m_y;
    }
};

Now, let's considerate a function foo that sums two Vec2 and compute the dot product with a third Vec2.

In [16]:
float foo(const Vec2& A, const Vec2& B)
{
    Vec2 C(2.0f, 3.0f);

    //Here is called the ctor of vec2 to build a Vec2 and
    //the result of A + B is copied, however this freshly
    //created Vec2 is destroied at the end of this call
    return (A + B).dot(C);
}

Using lazy evaluation, it is possible to save a call to Vec2 ctor&dtor because there is no need for Vec2::operator+ to be called.